# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5aebc57160>
├── 'a' --> tensor([[-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243]])
└── 'x' --> <FastTreeValue 0x7f5aebc572e0>
    └── 'c' --> tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                        [ 0.7825, -2.5640, -0.1113, -0.4586],
                        [ 0.9720,  0.0303,  0.3396, -1.2330]])

In [4]:
t.a

tensor([[-0.2689,  0.3690, -0.1727],
        [-1.9622, -1.2579,  0.1243]])

In [5]:
%timeit t.a

77 ns ± 0.0451 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5aebc57160>
├── 'a' --> tensor([[-3.7126, -1.6334,  0.6980],
│                   [-1.4830,  0.0943, -0.7372]])
└── 'x' --> <FastTreeValue 0x7f5aebc572e0>
    └── 'c' --> tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                        [ 0.7825, -2.5640, -0.1113, -0.4586],
                        [ 0.9720,  0.0303,  0.3396, -1.2330]])

In [7]:
%timeit t.a = new_value

80.3 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243]]),
    x: Batch(
           c: tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                      [ 0.7825, -2.5640, -0.1113, -0.4586],
                      [ 0.9720,  0.0303,  0.3396, -1.2330]]),
       ),
)

In [10]:
b.a

tensor([[-0.2689,  0.3690, -0.1727],
        [-1.9622, -1.2579,  0.1243]])

In [11]:
%timeit b.a

68 ns ± 0.0295 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4086,  0.8293,  0.0233],
               [-0.0268, -0.4543, -1.5669]]),
    x: Batch(
           c: tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                      [ 0.7825, -2.5640, -0.1113, -0.4586],
                      [ 0.9720,  0.0303,  0.3396, -1.2330]]),
       ),
)

In [13]:
%timeit b.a = new_value

542 ns ± 0.0497 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

941 ns ± 0.334 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 32.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 596 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5aebc7b3d0>
├── 'a' --> tensor([[[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]],
│           
│                   [[-0.2689,  0.3690, -0.1727],
│                    [-1.9622, -1.2579,  0.1243]]])
└── 'x' --> <FastTreeValue 0x7f5a23fea7c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 73.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5a1be12e80>
├── 'a' --> tensor([[-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243],
│                   [-0.2689,  0.3690, -0.1727],
│                   [-1.9622, -1.2579,  0.1243]])
└── 'x' --> <FastTreeValue 0x7f5a23f50850>
    └── 'c' --> tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                        [ 0.7825, -2.5640, -0.1113, -0.4586],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 57.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

81.3 µs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]],
       
               [[-0.2689,  0.3690, -0.1727],
                [-1.9622, -1.2579,  0.1243]]]),
    x: Batch(
           c: tensor([[[ 0.5554,  0.0865,  1.1706,  0.6859],
                       [ 0.7825, -2.5640, -0.1113, -0.4586],
                       [ 0.9720,  0.0303,  0.3396, -1.2330]],
         

In [26]:
%timeit Batch.stack(batches)

84.2 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243],
               [-0.2689,  0.3690, -0.1727],
               [-1.9622, -1.2579,  0.1243]]),
    x: Batch(
           c: tensor([[ 0.5554,  0.0865,  1.1706,  0.6859],
                      [ 0.7825, -2.5640, -0.1113, -0.4586],
                      [ 0.9720,  0.0303,  0.3396, -1.2330],
                      [ 0.5554,  0.0865,  1.1706,  0.6859],
                      [ 0.7825, -

In [28]:
%timeit Batch.cat(batches)

154 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

349 µs ± 5.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
